In [2]:
import tensorflow as tf
import pandas as pd

Importing the dataset

In [4]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

Data and type checking

In [7]:
print(type(train))
train.head()

<class 'pandas.core.frame.DataFrame'>


,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


Breaking into parts

In [8]:
train_y = train.pop('Species')
test_y = test.pop('Species')

In [9]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [10]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


Creating the input function and feature columns

In [12]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle :
      ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

In [35]:
train_input_fn = make_input_fn(train, train_y, num_epochs=1000)
test_input_fn = make_input_fn(test, test_y, num_epochs=1)

In [18]:
fc = []
for i in train.keys():
  fc.append(tf.feature_column.numeric_column(key = i))

In [19]:
fc

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

Model Generation

In [45]:
classifier = tf.estimator.DNNClassifier(
    hidden_units = [10,30,10],
    n_classes = 3,
    feature_columns = fc,
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpqbb5auwm', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [46]:
classifier.train(train_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpqbb5auwm/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.1529391, step = 0
INFO:tensorflow:global_step/sec: 570.75
INFO:tensorflow:loss = 1.0250826, step = 100 (0.181 sec)
INFO:tensorflow:global_step/sec: 884.149
INFO:tensorflow:loss = 0.9791442, step = 200 (0.113 sec)
INFO:tensorflow:global_step/sec: 856.489
INFO:tensorflow:loss = 0.9489768, step = 300 (0.114 sec)
INFO:tensorflow:global_step/sec: 851.056
INFO:tensorflow:loss = 0.92656213, step = 400 (0.117 sec)
INFO:tensorflow:global_step/sec: 959.569
INFO:tensorflow:loss = 0.9080088, step = 500 (0.104 sec)
INFO:t

In [47]:
classifier.evaluate(test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-13T01:13:57Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpqbb5auwm/model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.25780s
INFO:tensorflow:Finished evaluation at 2021-01-13-01:13:58
INFO:tensorflow:Saving dict for global step 4000: accuracy = 0.8333333, average_loss = 0.56983066, global_step = 4000, loss = 0.56983066
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4000: /tmp/tmpqbb5auwm/model.ckpt-4000


{'accuracy': 0.8333333,
 'average_loss': 0.56983066,
 'global_step': 4000,
 'loss': 0.56983066}

Analyzing results

In [48]:
a = list(classifier.predict(test_input_fn))
len(a)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpqbb5auwm/model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


30

In [49]:
a[0]

{'all_class_ids': array([0, 1, 2], dtype=int32),
 'all_classes': array([b'0', b'1', b'2'], dtype=object),
 'class_ids': array([1]),
 'classes': array([b'1'], dtype=object),
 'logits': array([-0.59681475,  0.52780455,  0.4756859 ], dtype=float32),
 'probabilities': array([0.14282198, 0.43975517, 0.41742274], dtype=float32)}

In [50]:
print("Real \t Prediction")
for i in range(30):
  print(test_y[i],"\t",a[i]['class_ids'][0])

Real 	 Prediction
1 	 1
2 	 2
0 	 0
1 	 1
1 	 2
1 	 1
0 	 0
2 	 2
1 	 1
2 	 2
2 	 2
0 	 0
2 	 2
1 	 2
1 	 1
0 	 0
1 	 1
0 	 0
0 	 0
2 	 2
0 	 0
1 	 2
2 	 2
1 	 2
1 	 1
1 	 2
0 	 0
1 	 1
2 	 2
1 	 1
